In [50]:
import cv2
import pandas as pd
import numpy as np
import os
from skimage.feature.texture import graycomatrix
from skimage.feature.texture import graycoprops
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [51]:
def get_feature(img):
    
    img_graymatrix = graycomatrix(img, [1], [0, np.pi/2])
    # print("img_graymatrix shape: ",img_graymatrix.shape)
    
    img_contrast = graycoprops(img_graymatrix, 'contrast')
    # print("img_contrast shape: ", img_contrast.shape)
    
    img_homogeneity = graycoprops(img_graymatrix, 'homogeneity')
    # print("img_homogeneity shape: ", img_homogeneity.shape)
    
    img_correlation = graycoprops(img_graymatrix, 'correlation')
    # print("img_correlation shape: ", img_correlation.shape)
    
    
    img_contrast_flattened = img_contrast.flatten()
    img_homogeneity_flattened = img_homogeneity.flatten()
    img_correlation_flattened = img_correlation.flatten()
    
    
    features = np.concatenate([img_contrast_flattened, img_homogeneity_flattened, 
                             img_correlation_flattened])
    # print("final_feature shape: ",features.shape)
    
    return(features)

In [52]:
def feature_extraction(path_to_folder, class_label):
    data_list=[]
#     count=1
    for file_name in tqdm(os.listdir(path_to_folder)):
#         if(count>1):
#             break
        path_to_img = os.path.join(path_to_folder,file_name)
        img = cv2.imread(path_to_img, 0) # grayscale image
         
        if np.shape(img) == ():
            continue
        
        final_feature = get_feature(img)
        # print("final_feature shape: ",final_feature.shape)
        # print("final_feature is: ",final_feature)
        
        final_feature=list(final_feature)
        final_feature.insert(0,file_name)
        final_feature.insert(1,class_label)
        
        data_list.append(final_feature)
        
#         count+=1
       
    return(data_list)

In [53]:
nonfog_path = "NONFOGGY_SVM"
fog_path = "FOGGY_SVM"



In [54]:
data_list1 = feature_extraction(nonfog_path, 0)
data_list2 = feature_extraction(fog_path, 1)




100%|██████████| 520/520 [00:12<00:00, 42.00it/s]


In [55]:
df1 = pd.DataFrame(data_list1)

df2 = pd.DataFrame(data_list2)

In [56]:
# Concatenate DataFrames along rows
df = pd.concat([df1, df2], ignore_index=True)

In [57]:
# Rename columns
df.rename(columns={0: "image_names", 1: "label"}, inplace=True)

# Save the DataFrame to a CSV file

df.to_csv('features.csv', index=False)


In [58]:
df.shape

(1040, 8)

SVM CLASSIFIER

In [59]:
# Getting all values from the csv file in the form of array
array=df.values
# print(array)
# Extracting the features from 3: coloumn and for all rows start : End  
x_feature=array[:,2:]

#Extracting the labels from 1st coloumn only as integer 
y_label=array[:,1].astype('int')


print(x_feature.shape)
print(y_label.shape)

(1040, 6)
(1040,)


In [60]:
#Help Ous to split the x feature in train and test ,where test_size indicates 90 % is used for training and 10 % is used for testing 
# Random_state helps ous to to shuffle the data before splitting 
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,y_label,test_size=0.10,random_state=7)
# print(X_train,X_test)
# print(X_test)
# print(np.array(X_train[23]))


In [61]:
from sklearn.preprocessing import MinMaxScaler
# Normalise the data after splitting to avoid information leak between train and test set.

scaler_norm = MinMaxScaler()

# Helps ous to calculate the min max values for the features present in X_train .fit learns the min max of the training features 
# The same is applied on the testing Features 
X_train = scaler_norm.fit_transform(X_train)
X_test = scaler_norm.fit_transform(X_test)
# print(X_train)
# print(X_test)




In [62]:
# Random check
model_SVC=SVC(kernel='rbf',C=100,gamma=0.001)

kfold=KFold(n_splits=10, shuffle=True)
cv_results=cross_val_score(model_SVC,X_train,Y_train,cv=kfold,scoring='accuracy')
msg="%s %f (%f)" % ('Training Accuracy: ',cv_results.mean(),cv_results.std())
print(msg)

Training Accuracy:  0.812011 (0.027523)


In [63]:
# SVM hypertuning using GridSeachCV
model_SVC=SVC()

kfold=KFold(n_splits=10)
param_grid = {'C': [1, 10, 100, 500, 1000], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']} 

grid=GridSearchCV(estimator=model_SVC,param_grid=param_grid,scoring='accuracy',cv=kfold,verbose=3)
grid_result=grid.fit(X_train,Y_train)

print("Best: %f using %s" % (grid_result.best_score_,grid_result.best_params_))

Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV 1/10] END .........C=1, gamma=1, kernel=rbf;, score=0.787 total time=   0.0s
[CV 2/10] END .........C=1, gamma=1, kernel=rbf;, score=0.883 total time=   0.0s
[CV 3/10] END .........C=1, gamma=1, kernel=rbf;, score=0.883 total time=   0.0s
[CV 4/10] END .........C=1, gamma=1, kernel=rbf;, score=0.787 total time=   0.0s
[CV 5/10] END .........C=1, gamma=1, kernel=rbf;, score=0.862 total time=   0.0s
[CV 6/10] END .........C=1, gamma=1, kernel=rbf;, score=0.872 total time=   0.0s
[CV 7/10] END .........C=1, gamma=1, kernel=rbf;, score=0.871 total time=   0.0s
[CV 8/10] END .........C=1, gamma=1, kernel=rbf;, score=0.892 total time=   0.0s
[CV 9/10] END .........C=1, gamma=1, kernel=rbf;, score=0.817 total time=   0.0s
[CV 10/10] END ........C=1, gamma=1, kernel=rbf;, score=0.774 total time=   0.0s
[CV 1/10] END .......C=1, gamma=0.1, kernel=rbf;, score=0.787 total time=   0.0s
[CV 2/10] END .......C=1, gamma=0.1, kernel=rb

In [64]:
model_SVC = SVC(C=100,gamma=1, kernel='rbf')
model_SVC.fit(X_train,Y_train) 
predictions=model_SVC.predict(X_test)
# print(len(predictions))
print("PREDICTED VALUE :" ,predictions[1:15:1])
print("GROUND TRUTH VALUE :" ,Y_test[1:15:1])
# 1 -- Foggy
# 0 -- NON Foggy

PREDICTED VALUE : [0 1 0 0 1 0 1 1 0 1 0 0 1 1]
GROUND TRUTH VALUE : [0 1 0 1 1 0 1 1 0 1 0 0 0 1]


# TO check whether the output predicted is correct or not 

In [49]:

from sklearn.preprocessing import MinMaxScaler
import cv2
import joblib
nonfog_path1 = "NON_FOGTEST_SVM"
data_list3 = feature_extraction(nonfog_path1, 0)
df3 = pd.DataFrame(data_list3)
df3.shape
array1=df3.values
img_feature=array1[:,2:]

#Extracting the labels from 1st coloumn only as integer 
y_label1=array1[:,1].astype('int')
print(img_feature.shape)
print(y_label1.shape)
# Normalise the data after splitting to avoid information leak between train and test set.

scaler_norm1 = MinMaxScaler()
img_feature = scaler_norm1.fit_transform(img_feature)
model_SVC = joblib.load('svm_model.joblib')
pred=model_SVC.predict(img_feature)
print(pred)
img = cv2.imread(r"A:\MINI_PROJECT_1\Train\cloudy\CLEANN 13.jpg")
img = cv2.resize(img,(500,500))
# Display the image in a window named "Image"
cv2.imshow("Image", img)

# Wait for a key press and then close the window
cv2.waitKey(0)
cv2.destroyAllWindows()

100%|██████████| 1/1 [00:00<00:00, 18.11it/s]


(1, 6)
(1,)
[1]


In [ ]:
import joblib
# Save the trained model to a file using joblib
joblib.dump(model_SVC, 'svm_model.joblib')

# CONFUSION MATRIX
                                      predicted
                         fog  non fog          fog   non fog
Actual            fog    [[TN, FP]             [[ 8      0]
              non fog     [FN, TP]]             [ 1     34]]


TN (True Negative): Instances that are correctly predicted as negative.
FP (False Positive): Instances that are incorrectly predicted as positive.
FN (False Negative): Instances that are incorrectly predicted as negative.
TP (True Positive): Instances that are correctly predicted as positive.

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print("Test Accuracy Of our model : ",accuracy_score(Y_test,predictions))

# print("Accuracy Of our model is: {:.4f}".format(accuracy_score(Y_test, predictions)))

print("Confusion Matrix of our model is :")
print(confusion_matrix(Y_test,predictions))
print(classification_report(Y_test,predictions))

# CLASSIFICATION REPORT
precision = TP/(TP+FP)

recall = TP/(TP+FN)

f1-score = 2*P*R/(P+R)

support --> for those many samples 

accuracy = How crrct prictions our model will do

macro avg = P1 + P2/2

weighted avg = P1 * samples + P2 * samples/ total samples